#### RAMP Summer 2024 1 - Cloud storage
Level 1 The cloud storage system should support operations to add files, copy files, and get files stored on the system. ADD_FILE <name> <size> — should add a new file name to the storage. size is the amount of memory required in bytes. The current operation fails if a file with the same name already exists. Returns "true" if the file was added successfully or "false" otherwise. COPY_FILE <nameFrom> <nameTo> — should copy the file at nameFrom to nameTo. The operation fails if nameFrom points to a file that does not exist or points to a directory. The operation fails if the specified file already exists at nameTo. Returns "true" if the file was copied successfully or "false" otherwise. GET_FILE_SIZE <name> — should return a string representing the size of the file name if it exists, or an empty string otherwise.
Level 4
Implement support for file compression.

COMPRESS_FILE <userId> <name> — should compress the file name if it belongs to userId. The compressed file should be replaced with a new file named <name>.COMPRESSED. The size of the newly created file should be equal to the half of the original file. The size of all files is guaranteed to be even, so there should be no fractional calculations. It is also guaranteed that name for this operation never points to a compressed file - i.e., it never ends with .COMPRESSED. Compressed files should be owned by userId — the owner of the original file. Returns a string representing the remaining storage capacity for userId if the file was compressed successfully or an empty string otherwise.
Note that because file names can only contain lowercase letters, compressed files cannot be added via ADD_FILE.
It is guaranteed that all COPY_FILE operations will preserve the suffix .COMPRESSED.

DECOMPRESS_FILE <userId> <name> — should revert the compression of the file name if it belongs to userId. It is guaranteed that name for this operation always ends with .COMPRESSED. If decompression results in the userId exceeding their storage capacity limit or a decompressed version of the file with the given name already exists, the operation fails. Returns a string representing the remaining capacity of userId if the file was decompressed successfully or an empty string otherwise.

In [1]:
def solution(queries):
    cloud_storage = {}
    result = []
    users = {}
    
    users["admin"]={"capacity": float("inf"), "files": cloud_storage}

    def add_file(cloud_storage, name, size):
        if name in cloud_storage:
            return "false"
        cloud_storage[name] = {"size": int(size)}
        return "true"

    def copy_file(users, name_from, name_to):
        
        for user_id, user in users.items():
            if name_from in user["files"] and "/" != name_from[:-1]:
                file_size = user["files"][name_from]
                if (
                    name_to not in user["files"]
                    and (user_id == "admin" or users[user_id]["capacity"] >= file_size)
                ):
                    user["files"][name_to] = file_size
                    if user_id != "admin":
                        users[user_id]["capacity"] -= file_size
                    return "true"
        return "false"
        
        # if name_from not in cloud_storage or "/" == name_from[:-1] or name_to in cloud_storage or "/" == name_from[:-1]:
            
        #     return "false"
        # cloud_storage[name_to] = {"size": cloud_storage[name_from]["size"]}
        # return "true"
    
    def get_file_size(users, filename):
        for user_id, user in users.items():
            if filename in user["files"]:
                if user_id != "admin":
                    size = users[user_id]["files"][filename]
                    return str(size)
        
                return str(users[user_id]["files"][filename]["size"])
        return ""
        
    def find_file(users, prefix, suffix):
        matching_files = []
        for user_id, user in users.items():
            for name, size in user["files"].items():
                if user_id != "admin":
                    if name.startswith(prefix) and name.endswith(suffix):
                        matching_files.append((name, size))
                else:
                    if name.startswith(prefix) and name.endswith(suffix):
                        matching_files.append((name, users[user_id]["files"][name]["size"]))
                    

        if not matching_files:
            return ""

        # Sort files in descending order of size, and lexicographically in case of ties
        sorted_files = sorted(matching_files, key=lambda x: (-x[1], x[0]))

        # Format the result string
        result_str = ", ".join(f"{name}({size})" for name, size in sorted_files)
        return result_str
    
    def add_user(users, user_id, capacity):
        if user_id in users:
            return "false"
        users[user_id] = {"capacity": int(capacity), "files": {}}
        return "true"
        
    def add_file_by(users, user_id, name, size):
        
        if user_id not in users or name in users[user_id]["files"] or users[user_id]["capacity"] < int(size) or any(name in user["files"] for user in users.values()):
            return ""
        
        users[user_id]["files"][name] = int(size)
        users[user_id]["capacity"] -= int(size)

        return str(users[user_id]["capacity"])

    def update_capacity(users, user_id, new_capacity):
        if user_id not in users:
            return ""
        
        # Remove files if the new capacity is exceeded
        current_capacity = users[user_id]["capacity"]
        new_capacity = int(new_capacity)
        total_size = sum(users[user_id]["files"].values())
        if total_size > new_capacity:

            
            files_to_remove = []
            remaining_capacity = new_capacity

            # Sort files lexicographically and by size
            sorted_files = sorted(users[user_id]["files"].items(), key=lambda x: (x[0], x[1]))
            for file, size in sorted_files:
                if remaining_capacity >= size:
                    remaining_capacity -= size
                else:
                    files_to_remove.append(file)

            # Remove files from user's storage
            for file in files_to_remove:
                current_capacity += users[user_id]["files"].pop(file)

            users[user_id]["capacity"] = new_capacity - sum(users[user_id]["files"].values())
            return str(len(files_to_remove))
        else:
            users[user_id]["capacity"] = new_capacity - sum(users[user_id]["files"].values())
            return "0"

    def compress_file(users, user_id, name):
        if user_id not in users or name not in users[user_id]["files"] or name.endswith(".COMPRESSED"):
            return ""
        if user_id != "admin":
            original_size = users[user_id]["files"][name]
        else:
            original_size = users[user_id]["files"][name]["size"]
        

        compressed_size = original_size // 2
        update_capacity(users, user_id, compressed_size)
        if name in users[user_id]["files"]:
            del users[user_id]["files"][name]
        return str(users[user_id]["capacity"])

    def decompress_file(users, user_id, name):
        if user_id not in users or not name.endswith(".COMPRESSED") or name in users[user_id]["files"]:
            return ""
        original_name = name[:-11]  # Remove ".COMPRESSED" suffix
        if original_name in users[user_id]["files"] or users[user_id]["capacity"] < users[user_id]["files"][name]:
            return ""
        compressed_size = users[user_id]["files"][name]
        users[user_id]["files"][original_name] = 2 * compressed_size
        users[user_id]["capacity"] += compressed_size
        del users[user_id]["files"][name]
        return str(users[user_id]["capacity"])
    
    for query in queries:
        operation, *params = query
        if operation == "ADD_FILE":
            result.append(add_file(cloud_storage, *params))
        elif operation == "COPY_FILE":
            result.append(copy_file(users, *params))
        elif operation == "GET_FILE_SIZE":
            result.append(get_file_size(users, *params))
        elif operation == "FIND_FILE":
            result.append(find_file(users, *params))
        elif operation == "ADD_USER":
            result.append(add_user(users, *params))
        elif operation == "ADD_FILE_BY":
            result.append(add_file_by(users, *params))
        elif operation == "UPDATE_CAPACITY":
            result.append(update_capacity(users, *params))
        elif operation == "COMPRESS_FILE":
            result.append(compress_file(users, *params))
        elif operation == "DEPRESS_FILE":
            result.append(decompress_file(users, *params))
        

    return result


queries = [["ADD_USER","username","1000"], 
 ["ADD_FILE_BY","username","/my/super/file.txt","100"], 
 ["COMPRESS_FILE","username","/my/super/file.txt"], 
 ["GET_FILE_SIZE","/my/super/file.txt"], 
 ["GET_FILE_SIZE","/my/super/file.txt.COMPRESSED"]
 ]
print(solution(queries))

['true', '900', '50', '', '']


#### RAMP Summer 2024  - Bank

In [3]:
def solution(queries):
    accounts = {}
    transfers = {}
    transfer_counter = 0
    MILLISECONDS_IN_1_DAY = 86400000


    def create_account(timestamp, account_id):
        if account_id not in accounts:
            accounts[account_id] = {"balance": 0, "transactions": [], "transfers": {}}
            return "true"
        else:
            return "false"

    def deposit(timestamp, account_id, amount):
        if account_id in accounts:
            amount = int(amount)
            
            # if accounts[account_id]: 
            #     if accounts[account_id]["balance"]:
            #         print( accounts[account_id]["balance"] )

            accounts[account_id]["balance"] += amount
            accounts[account_id]["transactions"].append({"type": "DEPOSIT", "amount": amount, "timestamp": int(timestamp)})
            
            
            return str(accounts[account_id]["balance"])
        else:
            return ""

    def pay(timestamp, account_id, amount):
        if account_id in accounts:
            amount = int(amount)
            if accounts[account_id]["balance"] >= amount:
                accounts[account_id]["balance"] -= amount
                accounts[account_id]["transactions"].append({"type": "PAY", "amount": amount, "timestamp": int(timestamp)})
                return str(accounts[account_id]["balance"])
            else:
                return ""
        else:
            return ""
    
    def top_activity(timestamp, n):
            sorted_accounts = sorted(accounts.items(), key=lambda x: (-sum(t['amount'] for t in x[1]['transactions']), x[0]), reverse=False)
            
            top_n_accounts = sorted_accounts[:n]
            result = ", ".join(f"{account_id}({sum(t['amount'] for t in account_data['transactions'])})" for account_id, account_data in top_n_accounts)
            return result

    def transfer(timestamp, source_account_id, target_account_id, amount):
        nonlocal transfer_counter
        if source_account_id == target_account_id or source_account_id not in accounts or target_account_id not in accounts:
            return ""

        amount = int(amount)
        if accounts[source_account_id]["balance"] < amount:
            return ""

        transfer_id = f"transfer{transfer_counter + 1}"
        transfer_counter += 1

        transfers[transfer_id] = {
            "source_account": source_account_id,
            "target_account": target_account_id,
            "amount": amount,
            "timestamp": int(timestamp),
            "accepted": False
        }

        accounts[source_account_id]["balance"] -= amount
        accounts[source_account_id]["transfers"][transfer_id] = {"type": "TRANSFER_OUT", "amount": amount, "timestamp": int(timestamp)}

        accounts[target_account_id]["transfers"][transfer_id] = {"type": "TRANSFER_IN", "amount": amount, "timestamp": int(timestamp)}

        return transfer_id
    
    def accept_transfer(timestamp, account_id, transfer_id):
      
        if (transfer_id not in transfers 
        or transfers[transfer_id]["accepted"] 
        #expired
        or int(timestamp) > transfers[transfer_id]["timestamp"] + MILLISECONDS_IN_1_DAY  
        
        or account_id != transfers[transfer_id]["target_account"]):
            if transfer_id in transfers:
                if int(timestamp) > transfers[transfer_id]["timestamp"] + MILLISECONDS_IN_1_DAY:
                    
                    source_account_id = transfers[transfer_id]["source_account"]
                    amount = transfers[transfer_id]["amount"]
                    accounts[source_account_id]["balance"] += amount

            
            return "false"
        else:
            source_account_id = transfers[transfer_id]["source_account"]
            
            
            
            
            amount = transfers[transfer_id]["amount"]

            transfers[transfer_id]["accepted"] = True
            
            accounts[account_id]["balance"] += amount
            
            accounts[account_id]["transactions"].append({"type": "TRANSFER_IN", "amount": amount, "timestamp": int(timestamp)})
            #? not sure + or - for out amount
            accounts[source_account_id]["transactions"].append({"type": "TRANSFER_OUT", "amount": amount, "timestamp": int(timestamp)})

            return "true"
    
    
    #MAIN
    output = []
    
    for query in queries:
        operation, timestamp, account_id, *args = query
        if operation == "CREATE_ACCOUNT":
            output.append(create_account(timestamp, account_id))
        elif operation == "DEPOSIT":
            output.append(deposit(timestamp, account_id, *args))
        elif operation == "PAY":
            output.append(pay(timestamp, account_id, *args))
        elif operation == "TOP_ACTIVITY":
            output.append(top_activity(timestamp, int(account_id)))
        elif operation == "TRANSFER":
            output.append(transfer(timestamp, account_id, *args))
        elif operation == "ACCEPT_TRANSFER":
            output.append(accept_transfer(timestamp, account_id, *args))
    return output

In [2]:
{ "input": { "queries": [ [ "CREATE_ACCOUNT", "1", "acc0" ], [ "CREATE_ACCOUNT", "2", "acc1" ], [ "CREATE_ACCOUNT", "3", "acc2" ], [ "CREATE_ACCOUNT", "4", "acc3" ], [ "CREATE_ACCOUNT", "5", "acc4" ], [ "CREATE_ACCOUNT", "6", "acc5" ], [ "CREATE_ACCOUNT", "7", "acc6" ], [ "CREATE_ACCOUNT", "8", "acc7" ], [ "CREATE_ACCOUNT", "9", "acc8" ], [ "CREATE_ACCOUNT", "10", "acc9" ], [ "DEPOSIT", "11", "acc0", "7757" ], [ "DEPOSIT", "12", "acc1", "8114" ], [ "DEPOSIT", "13", "acc2", "6692" ], [ "DEPOSIT", "14", "acc3", "5429" ], [ "DEPOSIT", "15", "acc4", "7852" ], [ "DEPOSIT", "16", "acc5", "6105" ], [ "DEPOSIT", "17", "acc6", "5747" ], [ "DEPOSIT", "18", "acc7", "8009" ], [ "DEPOSIT", "19", "acc8", "5165" ], [ "DEPOSIT", "20", "acc9", "5966" ], [ "PAY", "21", "acc0", "344" ], [ "PAY", "22", "acc1", "222" ], [ "PAY", "23", "acc2", "377" ], [ "PAY", "24", "acc3", "172" ], [ "PAY", "25", "acc4", "251" ], [ "PAY", "26", "acc5", "497" ], [ "PAY", "27", "acc6", "472" ], [ "PAY", "28", "acc7", "103" ], [ "PAY", "29", "acc8", "171" ], [ "PAY", "30", "acc9", "448" ], [ "TRANSFER", "31", "acc6", "acc0", "1358" ], [ "TRANSFER", "32", "acc0", "acc1", "1150" ], [ "TRANSFER", "33", "acc3", "acc2", "1235" ], [ "TRANSFER", "34", "acc0", "acc3", "1539" ], [ "TRANSFER", "35", "acc2", "acc4", "1253" ], [ "TRANSFER", "36", "acc2", "acc5", "1397" ], [ "TRANSFER", "37", "acc5", "acc6", "1861" ], [ "TRANSFER", "38", "acc2", "acc7", "1518" ], [ "TRANSFER", "39", "acc3", "acc8", "1635" ], [ "TRANSFER", "40", "acc1", "acc9", "1669" ], [ "TOP_ACTIVITY", "41", "10" ], [ "DEPOSIT", "86400041", "acc0", "506" ], [ "DEPOSIT", "86400042", "acc1", "276" ], [ "DEPOSIT", "86400043", "acc2", "361" ], [ "DEPOSIT", "86400044", "acc3", "757" ], [ "DEPOSIT", "86400045", "acc4", "129" ], [ "DEPOSIT", "86400046", "acc5", "477" ], [ "DEPOSIT", "86400047", "acc6", "676" ], [ "DEPOSIT", "86400048", "acc7", "754" ], [ "DEPOSIT", "86400049", "acc8", "873" ], [ "DEPOSIT", "86400050", "acc9", "242" ], [ "ACCEPT_TRANSFER", "86400051", "acc0", "transfer1" ], [ "ACCEPT_TRANSFER", "86400052", "acc1", "transfer2" ], [ "ACCEPT_TRANSFER", "86400053", "acc2", "transfer3" ], [ "ACCEPT_TRANSFER", "86400054", "acc3", "transfer4" ], [ "ACCEPT_TRANSFER", "86400055", "acc4", "transfer5" ], [ "ACCEPT_TRANSFER", "86400056", "acc5", "transfer6" ], [ "ACCEPT_TRANSFER", "86400057", "acc6", "transfer7" ], [ "ACCEPT_TRANSFER", "86400058", "acc7", "transfer8" ], [ "ACCEPT_TRANSFER", "86400059", "acc8", "transfer9" ], [ "ACCEPT_TRANSFER", "86400060", "acc9", "transfer10" ], [ "TOP_ACTIVITY", "86400061", "10" ], [ "PAY", "86400062", "acc0", "348" ], [ "PAY", "86400063", "acc1", "166" ], [ "PAY", "86400064", "acc2", "281" ], [ "PAY", "86400065", "acc3", "267" ], [ "PAY", "86400066", "acc4", "421" ], [ "PAY", "86400067", "acc5", "387" ], [ "PAY", "86400068", "acc6", "172" ], [ "PAY", "86400069", "acc7", "427" ], [ "PAY", "86400070", "acc8", "481" ], [ "PAY", "86400071", "acc9", "459" ], [ "TRANSFER", "86400073", "acc9", "acc0", "1938" ], [ "TRANSFER", "86400074", "acc3", "acc1", "1764" ], [ "TRANSFER", "86400075", "acc7", "acc2", "1660" ], [ "TRANSFER", "86400076", "acc4", "acc3", "1541" ], [ "TRANSFER", "86400077", "acc8", "acc4", "1379" ], [ "TRANSFER", "86400078", "acc2", "acc5", "1219" ], [ "TRANSFER", "86400079", "acc5", "acc6", "1519" ], [ "TRANSFER", "86400080", "acc2", "acc7", "1832" ], [ "TRANSFER", "86400081", "acc6", "acc8", "1645" ], [ "TRANSFER", "86400082", "acc2", "acc9", "1960" ], [ "ACCEPT_TRANSFER", "172800083", "acc0", "transfer11" ], [ "ACCEPT_TRANSFER", "172800084", "acc1", "transfer12" ], [ "ACCEPT_TRANSFER", "172800085", "acc2", "transfer13" ], [ "ACCEPT_TRANSFER", "172800086", "acc3", "transfer14" ], [ "ACCEPT_TRANSFER", "172800087", "acc4", "transfer15" ], [ "ACCEPT_TRANSFER", "172800088", "acc5", "transfer16" ], [ "ACCEPT_TRANSFER", "172800089", "acc6", "transfer17" ], [ "ACCEPT_TRANSFER", "172800090", "acc7", "transfer18" ], [ "ACCEPT_TRANSFER", "172800091", "acc8", "transfer19" ], [ "ACCEPT_TRANSFER", "172800092", "acc9", "transfer20" ], [ "TOP_ACTIVITY", "172800094", "10" ] ] }, "output": [ "true", "true", "true", "true", "true", "true", "true", "true", "true", "true", "7757", "8114", "6692", "5429", "7852", "6105", "5747", "8009", "5165", "5966", "7413", "7892", "6315", "5257", "7601", "5608", "5275", "7906", "4994", "5518", "transfer1", "transfer2", "transfer3", "transfer4", "transfer5", "transfer6", "transfer7", "transfer8", "transfer9", "transfer10", "acc1(8336), acc7(8112), acc4(8103), acc0(8101), acc2(7069), acc5(6602), acc9(6414), acc6(6219), acc3(5601), acc8(5336)", "7919", "8168", "6676", "6014", "7730", "6085", "5951", "8660", "5867", "5760", "false", "false", "false", "false", "false", "false", "false", "false", "false", "false", "acc7(8866), acc1(8612), acc0(8607), acc4(8232), acc2(7430), acc5(7079), acc6(6895), acc9(6656), acc3(6358), acc8(6209)", "7571", "8002", "6395", "5747", "7309", "5698", "5779", "8233", "5386", "5301", "transfer11", "transfer12", "transfer13", "transfer14", "transfer15", "transfer16", "transfer17", "transfer18", "transfer19", "transfer20", "false", "false", "false", "false", "false", "false", "false", "false", "false", "false", "acc7(9293), acc0(8955), acc1(8778), acc4(8653), acc2(7711), acc5(7466), acc9(7115), acc6(7067), acc8(6690), acc3(6625)" ] }

{'input': {'queries': [['CREATE_ACCOUNT', '1', 'acc0'],
   ['CREATE_ACCOUNT', '2', 'acc1'],
   ['CREATE_ACCOUNT', '3', 'acc2'],
   ['CREATE_ACCOUNT', '4', 'acc3'],
   ['CREATE_ACCOUNT', '5', 'acc4'],
   ['CREATE_ACCOUNT', '6', 'acc5'],
   ['CREATE_ACCOUNT', '7', 'acc6'],
   ['CREATE_ACCOUNT', '8', 'acc7'],
   ['CREATE_ACCOUNT', '9', 'acc8'],
   ['CREATE_ACCOUNT', '10', 'acc9'],
   ['DEPOSIT', '11', 'acc0', '7757'],
   ['DEPOSIT', '12', 'acc1', '8114'],
   ['DEPOSIT', '13', 'acc2', '6692'],
   ['DEPOSIT', '14', 'acc3', '5429'],
   ['DEPOSIT', '15', 'acc4', '7852'],
   ['DEPOSIT', '16', 'acc5', '6105'],
   ['DEPOSIT', '17', 'acc6', '5747'],
   ['DEPOSIT', '18', 'acc7', '8009'],
   ['DEPOSIT', '19', 'acc8', '5165'],
   ['DEPOSIT', '20', 'acc9', '5966'],
   ['PAY', '21', 'acc0', '344'],
   ['PAY', '22', 'acc1', '222'],
   ['PAY', '23', 'acc2', '377'],
   ['PAY', '24', 'acc3', '172'],
   ['PAY', '25', 'acc4', '251'],
   ['PAY', '26', 'acc5', '497'],
   ['PAY', '27', 'acc6', '472'],
   ['PAY